# Binance Futures Aggregation Live Analysis V2.0

### In summary, this script connects to Binance's WebSocket API to receive aggregated trade data, processes and saves the data at regular intervals, and plots the volume comparison in real-time using Plotly.

In [ ]:
import websocket
import pandas as pd
import json
import threading
import time
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Initialize DataFrame and other variables
merged_df = pd.DataFrame(columns=['Volume_True', 'Volume_False', 'Vdiff'])
df_agg = pd.DataFrame(columns=["Symbol", "Price", "Quantity", "Is Buyer Market Maker"])
ws = None
save_interval = 300  # Save interval in seconds
graph1 = go.FigureWidget()

# Function to save and clear DataFrame
def save_and_clear_df():
    global merged_df
    while True:
        try:
            time.sleep(save_interval)  # Wait for the specified interval
            current_time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
            merged_df_temp = pd.DataFrame(columns=['Volume_True', 'Volume_False', 'Vdiff'])
            df_aggregation = df_agg.copy(deep=True)
            print(f"(Agg) Chapter Saved at {current_time}")
            df_agg.drop(df_agg.index, inplace=True)
            print(f"(Agg) Clearing Dataframe at {current_time}")
            df_aggregation.index = pd.to_datetime(df_aggregation.index)
            df_aggregation['Volume'] = df_aggregation['Price'] * df_aggregation['Quantity']
            df_true = df_aggregation.loc[df_aggregation['Is Buyer Market Maker'] == True]['Volume'].sum()
            df_false = df_aggregation.loc[df_aggregation['Is Buyer Market Maker'] == False]['Volume'].sum()
            vdiff = df_false - df_true
            merged_df_temp.loc[pd.Timestamp.now()] = [df_true, df_false, vdiff]
            merged_df = pd.concat([merged_df, merged_df_temp])
            plot_merged_df(merged_df)
        except Exception as e:
            print("An error occurred in saving and clearing DataFrame:", e)

# Create a thread to save and clear DataFrame
save_thread = threading.Thread(target=save_and_clear_df)
save_thread.daemon = True
save_thread.start()

# Function to plot merged DataFrame
def plot_merged_df(merged_df):
    try:
        # Add line plots for Volume_True and Volume_False
        trace_volume_true = go.Scatter(x=merged_df.index, y=merged_df['Volume_True'], mode='lines', name='Volume_True', line=dict(color='blue'))
        trace_volume_false = go.Scatter(x=merged_df.index, y=merged_df['Volume_False'], mode='lines', name='Volume_False', line=dict(color='orange'))

        # Create trace for Vdiff(False-True) as a bar chart
        bar_colors = ['red' if diff < 0 else 'green' for diff in merged_df['Vdiff'].fillna(0)]
        trace_vdiff = go.Bar(x=merged_df.index, y=merged_df['Vdiff'], name='Vdiff', marker=dict(color=bar_colors))

        # Create layout for the plot
        layout = go.Layout(
            title='Volume Comparison',
            yaxis=dict(title='Volume'),
            hovermode='x',
            width=2400, 
            height=1000 
        )

        # Clear the previous output
        clear_output(wait=True)

        # Show the plot
        fig = go.Figure(data=[trace_volume_true, trace_volume_false, trace_vdiff], layout=layout)
        fig.show()
    except Exception as e:
        print("An error occurred in plotting merged DataFrame:", e)

# Function to handle WebSocket message
def on_message(ws, message):
    try:
        msg = json.loads(message)
        if msg["e"] == "aggTrade":
            stream_agg_trade(msg)
    except Exception as e:
        print("An error occurred in handling WebSocket message:", e)

# Function to handle WebSocket error
def on_error(ws, error):
    print("An error occurred in WebSocket connection:", error)

# Function to handle WebSocket close
def on_close(ws):
    print("WebSocket connection closed. Reconnecting...")
    time.sleep(1)  # Wait for a second before reconnecting
    websocket_thread = threading.Thread(target=run_websocket)
    websocket_thread.start()

# Function to handle WebSocket open
def on_open(ws):
    print("WebSocket connection opened.")

# Function to process aggregated trade data
def stream_agg_trade(msg):
    try:
        event_time = pd.to_datetime(msg["E"], unit='ms')
        symbol = msg["s"]
        price = float(msg["p"])
        quantity = float(msg["q"])
        is_buyer_market_maker = msg["m"]
        df_agg.loc[event_time] = [symbol, price, quantity, is_buyer_market_maker]
        print(f"AGGREGATION: Event Time: {event_time} | Symbol: {symbol} | Price: {price} | Quantity: {quantity} | Is Buyer Market Maker: {is_buyer_market_maker}", end="\r")
    except Exception as e:
        print("An error occurred in processing aggregated trade data:", e)

# Function to run WebSocket connection
def run_websocket():
    global ws
    websocket.enableTrace(False)
    while True:
        try:
            socket_url = "wss://fstream.binance.com/ws/btcusdt@aggTrade"
            ws = websocket.WebSocketApp(socket_url, on_message=on_message, on_error=on_error, on_close=on_close)
            ws.on_open = on_open
            ws.run_forever()
        except Exception as e:
            print("An error occurred in WebSocket connection:", e)
            time.sleep(1)  # Wait for a second before retrying

# Start WebSocket connection in a separate thread
websocket_thread = threading.Thread(target=run_websocket)
websocket_thread.start()
